In [1]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPool1D, Dropout, ReLU, Conv2DTranspose, Input, Add
from tensorflow.keras import Model

In [2]:
class Conv1DTranspose(Conv2DTranspose):
    def __init__(self, filters, kernel_size, stride, **kwargs):
        self.output_channels = filters
        super(Conv1DTranspose, self).__init__(
            filters=filters,
            kernel_size=(kernel_size, 1),
            strides=(stride, 1),
            **kwargs
        )

    def build(self, input_shape):
        super(Conv1DTranspose, self).build((input_shape[0], input_shape[1], 1, input_shape[2]))

    def __call__(self, x):
        data = K.expand_dims(x, axis=2)
        data = super(Conv1DTranspose, self).__call__(data)
        data = K.squeeze(data, axis=2)
        return data
        
    def call(self, x):
        x = super(Conv1DTranspose, self).call(x)
        return x


# FCSN Model

In [3]:
input_size = (320, 1024)   # Tensorflow uses the Channels-last format by default
n_classes  = 10

inputs = Input(input_size)

# Block 1
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool1D(pool_size=2, strides=2, padding="SAME")(x)


In [4]:
# Block 2
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool1D(pool_size=2, strides=2, padding="SAME")(x)


In [5]:
# Block 3
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=1024, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool1D(pool_size=2, strides=2, padding="SAME")(x)


In [ ]:
# Block 4
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool1D(pool_size=2, strides=2, padding="SAME")(x)

pool4 = x


In [ ]:
# Block 5
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=2048, kernel_size=3, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool1D(pool_size=2, strides=2, padding="SAME")(x)


In [ ]:
# Block 6
x = Conv1D(filters=4096, kernel_size=1, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.5)(x)

# Block 7
x = Conv1D(filters=4096, kernel_size=1, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.5)(x)

# Block 8
x = Conv1D(filters=n_classes, kernel_size=1, padding="SAME")(x)
x = BatchNormalization()(x)
x = ReLU()(x)

x = Conv1DTranspose(filters=n_classes, kernel_size=4, padding="SAME", stride=2, use_bias=False)(x)

upscore = x

In [ ]:
score_pool = Conv1D(filters=n_classes, kernel_size=1, padding="SAME")(pool4)
score_pool = BatchNormalization()(score_pool)

x = Add()([upscore, score_pool])
outputs = Conv1DTranspose(filters=n_classes, kernel_size=16, padding="SAME", stride=16, use_bias=False)(x)

model = Model(inputs=inputs, outputs=outputs, name="FCSN")

In [28]:
print(x)

Tensor("Squeeze_7:0", shape=(None, 320, 10), dtype=float32)
